In [1]:
## DESCRIPTION ##
# ECDF temperature = meteo_temperature_AA_ECDFoverTime
# ECDF precipitation = meteo_precipitation_AA_ECDFoverTime
# Spatial boxplot bias kr11 - rea temperature = meteo_temperature_AA_boxplotSpatialBias_kr11-rea
# Spatial boxplot bias kr11 - rea precipitation = meteo_precipitation_AA_boxplotSpatialBias_kr11-rea

# Temporal boxplot bias kr11 - rea temperature = meteo_passirio_temperature_boxplotTemporalBias_kr11-rea (monthly)
# Temporal boxplot bias kr11 - rea temperature = meteo_passirio_temperature_boxplotTemporalBias_kr11-rea (hourly)
# Temporal boxplot bias kr11 - rea temperature = meteo_passirio_temperature_boxplotHourlyTemporalBias_kr11-rea (seasonal)
# Temporal boxplot bias kr11 - rea precipitation = meteo_passirio_precipitation_boxplotTemporalBias_kr11-rea (monthly)

In [2]:
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(sys.path[0]),'lib'))

from lib import *
current = DataCollector()

In [3]:
## SETUP
start_date_str = "2010-01-01T00:00:00"
end_date_str = "2019-12-31T23:00:00"
start_date = dt.datetime.strptime( start_date_str, '%Y-%m-%dT%H:%M:%S' )
end_date = dt.datetime.strptime( end_date_str, '%Y-%m-%dT%H:%M:%S' )

dates = pd.date_range(start_date, end_date, freq='h')
dates_days = pd.date_range(start_date, end_date, freq='d')

IDs_cells  = [115, 154, 174, 203, 207, 208]   # id grid points to compare meteo stations

In [4]:
## KRIGING 11X8 DATASET ##
path_kriging11_cells = 'D:\\hydrology\\projects\\era5_bias\\OLD\\comparison\\scripts\\kriging\\AltoAdige\\TMEAN\\' 
          
temperature_df_kr11 = pd.DataFrame()

for i,id_cell in enumerate(IDs_cells):
    data_tot = pd.read_csv( path_kriging11_cells + str(id_cell) + '.csv', header=0, index_col=0, \
        skiprows=4, parse_dates=True, names=['values'])
    data_tot = data_tot[start_date:end_date]
    data_tot[data_tot == -999] = None
    temperature_df_kr11 = pd.concat([temperature_df_kr11, data_tot],axis=1)

temperature_df_kr11_daily_mean = temperature_df_kr11.resample('d').mean()
temperature_df_kr11_monthly_mean = temperature_df_kr11.resample('MS').mean()

In [5]:
## REANALYSIS 11X8 DATASET ##
path_rea_cells = 'D:\\data\\GFS_models\\ECMWF\\ERA5Land-reanalysis\\t2m\\' 
          
temperature_df_rea = pd.DataFrame()

for i,id_cell in enumerate(IDs_cells):
    data_tot = pd.read_csv( path_rea_cells + str(id_cell) + '.csv', header=0, index_col=0, \
        skiprows=4, parse_dates=True, names=['values'])
    data_tot = data_tot[start_date:end_date]
    data_tot[data_tot == -999] = None
    temperature_df_rea = pd.concat([temperature_df_rea, data_tot],axis=1)

temperature_df_rea_daily_mean = temperature_df_rea.resample('d').mean()
temperature_df_rea_monthly_mean = temperature_df_rea.resample('MS').mean()

In [7]:
## OBSERVED DATASET ##
path_meteo_stations = 'D:\\data\\AA_weather_data\\'
ID_stations = [13,16,21,101,191,193]
ID_label= ['Alpe_di_Siusi_Zallinger(1920slm)', \
            'Anterselva_di_Sopra(1320slm)', \
            'Bolzano(254slm)', \
            'Merano(333slm)', \
            'Selva_di_Val_Gardena(1570slm)', \
            'Silandro(718slm)']

temperature_df_meteo_stations = pd.DataFrame()

for i,id_station in enumerate(ID_stations):
    data_tot = pd.read_csv( path_meteo_stations + 'temperature\\' + str(id_station) + '.txt', header=0, index_col=0, \
        skiprows=4, parse_dates=True, names=['values'])
    data_tot = data_tot[start_date:end_date]
    data_tot[data_tot == -999] = None
    temperature_df_meteo_stations = pd.concat([temperature_df_meteo_stations, data_tot],axis=1)

temperature_df_meteo_stations_daily_mean = temperature_df_meteo_stations.resample('d').mean()
temperature_df_meteo_stations_monthly_mean = temperature_df_meteo_stations.resample('MS').mean()

In [19]:
# bigframe1 = pd.DataFrame(index=IDs_cells,columns=['rmse','cc','mae'])
# bigframe2 = pd.DataFrame(index=IDs_cells,columns=['rmse','cc','mae'])
# bigframe3 = pd.DataFrame(index=IDs_cells,columns=['rmse','cc','mae'])
# bigframe4 = pd.DataFrame(index=IDs_cells,columns=['rmse','cc','mae'])

# for i,id_cell in enumerate(IDs_cells):
#     ##COEFF STATISTICI kriging vs reanalysis
#     Ndata = len(temperature_df_kr11_monthly_mean) 
#     rmse = np.sqrt((1/Ndata) * np.sum((temperature_df_kr11_monthly_mean.values - temperature_df_rea_monthly_mean.values)**2 ))       
#     cc  =  np.corrcoef(temperature_df_kr11_monthly_mean['values'],temperature_df_rea_monthly_mean['values'])
#     mae = ( np.sum( np.abs(temperature_df_kr11_monthly_mean['values'] - temperature_df_rea_monthly_mean['values']) )) / Ndata

#     bigframe1.loc[id_cell]['rmse'] = rmse
#     bigframe1.loc[id_cell]['cc'] = cc[i][1]
#     bigframe1.loc[id_cell]['mae'] = mae[i]

In [ ]:
for var in meteo_var:

    for var_r in meteo_var_r:
        
        if var=='TMEAN' and var_r=='t2m':

            #-------------------------------Grafici ECDF TEMPERATURA--------------------------------------------
            # DataFrame per l'ECDF
            df_k=pd.DataFrame() # dataframe kriging with all data in the directory
            metadata_k=pd.DataFrame() # metadata of the df_k files

            spatial_mean_k=pd.DataFrame(columns= ['dates','values']) # spatial mean kriging
            s_sort_k=pd.DataFrame(columns=['values']) # spatial mean kriging sorted
            s_frame_k=pd.DataFrame() # spatial ordeblack mean kriging
            e_s_k=pd.DataFrame()
            a_s_k=pd.DataFrame(columns=['values'])
            
            t_frame_k=pd.DataFrame() # temporal mean kriging ECDF

            season_k=pd.DataFrame() ### ?

            temporal_mean_k=pd.DataFrame() # temporal mean kriging
            t_sort_k=pd.DataFrame() # temporal sorted mean kriging
            e_t_k=pd.DataFrame()
            a_t_k=pd.DataFrame(columns=['values'])

            #raster_k=pd.DataFrame()  # media temporale kriging

            ### 

            df=pd.DataFrame() # dataframe reanalysis with all data in the directory
            metadata=pd.DataFrame() # metadata of the df files

            spatial_mean=pd.DataFrame(columns= ['dates','values'])  # spatial mean reanalysis
            s_sort=pd.DataFrame(columns=['values']) # spatial mean reanalysis sorted
            s_frame=pd.DataFrame(columns=['values','ecdf']) # spatial ordeblack mean reanalysis
            e_s=pd.DataFrame(columns=['ecdf'])
            a_s=pd.DataFrame(columns=['ecdf'])
            
            t_frame=pd.DataFrame() # temporal mean reanalysis ECDF

            season=pd.DataFrame() ### ?

            temporal_mean=pd.DataFrame() # temporal mean reanalysis
            t_sort=pd.DataFrame() # temporal sorted mean reanalysis
            e_t=pd.DataFrame()
            a_t=pd.DataFrame(columns=['values'])

            #raster=pd.DataFrame() # media temporale reanalysis

            diff_t=pd.DataFrame()
            #

            import glob
            ### assemblo la matrice per fare le medie spaziali e temporali - kriging
            f = path_kr +  'TMEAN/'
            file_tot = glob.glob(f+'*.csv')
            for file in file_tot:
                data_tot = pd.read_csv( file, index_col=0, parse_dates=True, skiprows=4, names=['values'] )
                data_tot_metadata = pd.read_csv( file, index_col=0, parse_dates=True, names=['values'] ) 
                data_tot = data_tot[t0:t1]
                data_tot_metadata = data_tot_metadata[0:3]
                #data_tot[data_tot == -999] = None
                #data_tot = data_tot.resample('d').mean()
                df_k=pd.concat([df_k,data_tot],axis=1)
                metadata_k=pd.concat([metadata_k,data_tot_metadata],axis=1)           
            
            ### assemblo la matrice per fare le medie spaziali e temporali - Reanalysis
            f=path_rea  + 't2m/'
            file_tot = glob.glob(f+'*.csv')
            for file in file_tot:
                data_tot = pd.read_csv(file,index_col=0,parse_dates=True,skiprows=4,names=['values'])
                data_tot_metadata = pd.read_csv(file,index_col=0,parse_dates=True,names=['values'])
                data_tot=data_tot[t0:t1]
                data_tot_metadata=data_tot_metadata[0:4]
                data_tot[data_tot == -999] = None
                #data_tot = data_tot.resample('d').mean()
                df=pd.concat([df,data_tot],axis=1)
                metadata=pd.concat([metadata,data_tot_metadata],axis=1)

            ### MEDIA SPAZIALE - Reanalysis           
            spatial_mean = df.mean(axis=1)
            # spatial_mean.to_csv(current.config["output_path"]+'media_spaziale_reanalysis_t2m_.csv',index=True) 
            s_frame = evaluateECDF( spatial_mean )
                    
            ## MEDIA TEMPORALE - Reanalysis
            season=df.resample('Y').mean()
            season=season.groupby(season.index.month).mean()
            season[season == 0] = None
            
            temporal_mean=df.mean(axis=0)
            t_frame = evaluateECDF( temporal_mean )
           
            ## MEDIA SPAZIALE - kriging          
            spatial_mean_k=df_k.mean(axis=1)
            s_frame_k = evaluateECDF( spatial_mean_k )

            ## MEDIA TEMPORALE - kriging 
            season_k=df_k.resample('Y').mean()                          ### ?
            season_k=season_k.groupby(season_k.index.month).mean()      ### ?
            season_k[season_k == 0] = None
            
            temporal_mean_k=df_k.mean(axis=0)   
            t_frame_k = evaluateECDF( temporal_mean_k )

            #----------------------------------Grafici MEDIE TEMPORALI------------------------------------------------------
            plots = []

            plt_conf = {}
            plt_conf["label"] = "Reanalysis 11x8"
            plots.append( (s_frame, plt_conf) )

            plt_conf = {}
            plt_conf["label"] = "Kriging 11x8"
            plots.append( (s_frame_k, plt_conf) )

            output_file = current.config["output_path"]current.config["output_path"] + "meteo\\temperature\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime.' + output_format
            createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
                scale_factor=0.5, my_dpi=50)

            output_file_hd = current.config["output_path"] + "meteo\\temperature\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_HD.' + output_format
            createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
                scale_factor=0.5, my_dpi=600)

            output_file = current.config["output_path"] + "meteo\\temperature\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_log.' + output_format
            createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
                xscale='log', scale_factor=0.5, my_dpi=50)

            output_file_hd = current.config["output_path"] + "meteo\\temperature\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_log_HD.' + output_format
            createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
                xscale='log', scale_factor=0.5, my_dpi=600)
           
            #----------------------------------Grafici MEDIE SPAZIALI------------------------------------------------------
           
            mean_r=pd.DataFrame()
            mean_k=pd.DataFrame()
            
            mean_r=df.mean(axis=1)
            mean_k=df_k.mean(axis=1)
            
            mean_r_avg = mean_r.groupby([mean_r.index.month, mean_r.index.day]).mean()
            mean_r_avg_365 = mean_r_avg.drop(mean_r_avg.index[59]) #cutoff the 29th of February
            mean_r_avg_365.reset_index(drop=True, inplace=True)

            mean_k_avg=mean_k.groupby([mean_k.index.month,mean_k.index.day]).mean()
            mean_k_avg_365=mean_k_avg.drop(mean_k_avg.index[59]) #cutoff the 29th of February
            mean_k_avg_365.reset_index(drop=True, inplace=True)

            year=pd.date_range(start='1/1/13', end='12/31/13')
            mean_r_avg_365.index=year
            mean_k_avg_365.index=year

            mean_r_month=mean_r.resample('MS').mean()
            mean_k_month=mean_k.resample('MS').mean()       
            mean_r_avg_365=mean_r_avg_365.resample('MS').mean()
            mean_k_avg_365=mean_k_avg_365.resample('MS').mean()            
            
            mean = mean_k_month - mean_r_month

            output_file = current.config["output_path"] + "meteo\\temperature\\alto_adige\\monthly\\meteo_" \
                + var_str + '_AA_monthly_boxplotSpatialBias_kr11-rea.' + output_format
            createBoxPlot( mean, "Time $[month]$", "Temperature bias $[\degree C]$", \
                output_file, output_format=output_format, scale_factor=0.5, my_dpi=50 )

            output_file_hd = current.config["output_path"] + "meteo\\temperature\\alto_adige\\monthly\\meteo_" \
                + var_str + '_AA_monthly_boxplotSpatialBias_kr11-rea_HD.' + output_format
            createBoxPlot( mean, "Time $[month]$", "Temperature bias $[\degree C]$", \
                output_file_hd, output_format=output_format, scale_factor=0.5, my_dpi=600 )

In [ ]:
for var in meteo_var:

    for var_r in meteo_var_r:

        if var=='P' and var_r=='ptot':
            
            for i,ID_cell in enumerate(IDs_cells):
                
                file=path_kr + var + '/' + str(ID_cell) + '.csv'
                file_r=path_rea  +var_r + '/hourly_ts/'  + str(ID_cell) + '.csv'
                meteo_var_ms='precipitation'
                station = ID_station[i] 
                station_label = ID_label[i] 
                file_ms=path_ms + meteo_var_ms + '/' + str(station) + '.txt'
                
                data = pd.read_csv(file,index_col=0,parse_dates=True, skiprows=4, names=['values'])
                data_r = pd.read_csv(file_r,index_col=0,parse_dates=True, skiprows=4, names=['values'])
                data_ms = pd.read_csv(file_ms,index_col=0,parse_dates=True, skiprows=4, names=['values'])
                
                data=data[ta:tb]
                data_r=data_r[t0:t1]
                data_ms=data_ms[ta:tb] 
                data[data == -999] = None
                data_r[data_r == -999] = None
                data_ms[data_ms == -999] = None                
                 
                data = data.resample('d').sum()  
                data_ms = data_ms.resample('d').sum()
                data_r=data_r.resample('d').sum()
                
                #converting dataframe in format float
                data = data.astype('float')
                data_r = data_r.astype('float')
            
            #-------------------------------Grafici ECDF PRECIPITAZIONE--------------------------------------------
            
            # DataFrame per l'ECDF
            df=pd.DataFrame()
            season=pd.DataFrame()
            spatial_mean=pd.DataFrame(columns= ['dates','values'])
            temporal_mean=pd.DataFrame()
            t_sort=pd.DataFrame()
            s_sort=pd.DataFrame()
            e_s=pd.DataFrame()
            a_s=pd.DataFrame(columns=['values'])
            e_t=pd.DataFrame()
            a_t=pd.DataFrame(columns=['values'])
            s_frame=pd.DataFrame()
            t_frame=pd.DataFrame()
            metadata=pd.DataFrame()
            raster=pd.DataFrame()
            df_k=pd.DataFrame()
            season_k=pd.DataFrame()
            spatial_mean_k=pd.DataFrame(columns= ['dates','values'])
            temporal_mean_k=pd.DataFrame()
            t_sort_k=pd.DataFrame()
            s_sort_k=pd.DataFrame()
            e_s_k=pd.DataFrame()
            a_s_k=pd.DataFrame(columns=['values'])
            e_t_k=pd.DataFrame()
            a_t_k=pd.DataFrame(columns=['values'])
            s_frame_k=pd.DataFrame()
            t_frame_k=pd.DataFrame()
            metadata_k=pd.DataFrame()
            raster_k=pd.DataFrame()
            diff_p=pd.DataFrame()
            #

            
            ### assemblo la matrice per fare le medie spaziali e temporali - Reanalysis
            f=path_rea  +  'ptot/hourly_ts/'
            file_tot = glob.glob(f+'*.csv')
            for file in file_tot:
                data_tot = pd.read_csv(file,index_col=0,parse_dates=True, skiprows=4,names=['values'])
                data_tot_metadata = pd.read_csv(file,index_col=0,parse_dates=True,names=['values'])
                data_tot=data_tot[t0:t1]
                data_tot_metadata=data_tot_metadata[0:4]
                data_tot[data_tot == -999] = None
                data_tot = data_tot.resample('d').sum()
                df=pd.concat([df,data_tot],axis=1)
                metadata=pd.concat([metadata,data_tot_metadata],axis=1)

            
            ### assemblo la matrice per fare le medie spaziali e temporali - kriging
            f=path_kr +'P/'
            file_tot = glob.glob(f+'*.csv')
            for file in file_tot:
                data_tot = pd.read_csv(file,index_col=0,parse_dates=True,skiprows=4,names=['values'])
                data_tot_metadata = pd.read_csv(file,index_col=0,parse_dates=True,names=['values'])
                data_tot=data_tot[t0:t1]
                data_tot_metadata=data_tot_metadata[0:3]
                data_tot[data_tot == -999] = None
                data_tot = data_tot.resample('d').sum()
                df_k=pd.concat([df_k,data_tot],axis=1)
                metadata_k=pd.concat([metadata_k,data_tot_metadata],axis=1)


            ### MEDIA SPAZIALE - Ranalysis           
            spatial_mean=df.mean(axis=1)
            s_frame = evaluateECDF( spatial_mean )

            ### MEDIA TEMPORALE - Reanalysis  
            season=df.resample('Y').sum()
            #season=df.resample('Q').sum()
            season=season.groupby(season.index.month).mean()
            season[season == 0] = None

            temporal_mean=df.mean(axis=0)   
            t_frame = evaluateECDF( temporal_mean )
           
            ### MEDIA SPAZIALE - kriging          
            spatial_mean_k=df_k.mean(axis=1)
            s_frame_k = evaluateECDF( spatial_mean_k )

            ### MEDIA TEMPORALE - kriging 
            season_k=df_k.resample('Y').sum()
            #season_k=df_k.resample('Q').sum()
            season_k=season_k.groupby(season_k.index.month).mean()
            season_k[season_k == 0] = None
            
            temporal_mean_k=df_k.mean(axis=0)
            t_frame_k = evaluateECDF( temporal_mean_k )

            plots = []

            plt_conf = {}
            plt_conf["label"] = "Reanalysis 11x8"
            plots.append( (s_frame, plt_conf) )

            plt_conf = {}
            plt_conf["label"] = "Kriging 11x8"
            plots.append( (s_frame_k, plt_conf) )

            output_file = current.config["output_path"] + "meteo\\precipitation\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime.' + output_format
            createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file, \
                scale_factor=0.5, my_dpi=50)

            output_file_hd =current.config["output_path"] + "meteo\\precipitation\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_HD.' + output_format
            createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file_hd, \
                scale_factor=0.5, my_dpi=600)
            
            output_file = current.config["output_path"] + "meteo\\precipitation\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_log.' + output_format
            createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file, \
                xscale='log', scale_factor=0.5, my_dpi=50)

            output_file_hd = current.config["output_path"] + "meteo\\precipitation\\alto_adige\\hourly\\meteo_" \
                + var_str + '_AA_hourly_ECDFoverTime_log_HD.' + output_format
            createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file_hd, \
                xscale='log', scale_factor=0.5, my_dpi=600)
      
            #----------------------------------Grafici MEDIE SPAZIALI------------------------------------------------------
           
            mean_r=pd.DataFrame()
            mean_k=pd.DataFrame()
            
            mean_r=df.mean(axis=1)
            mean_k=df_k.mean(axis=1)
            mean_r_month=mean_r.resample('MS').sum()
            mean_k_month=mean_k.resample('MS').sum()            
            
            mean_r_avg=mean_r.groupby([mean_r.index.month,mean_r.index.day]).mean()
            mean_r_avg_365=mean_r_avg.drop(mean_r_avg.index[59])
            mean_k_avg=mean_k.groupby([mean_k.index.month,mean_k.index.day]).mean()
            mean_k_avg_365=mean_k_avg.drop(mean_k_avg.index[59])
            mean_k_avg_365.reset_index(drop=True, inplace=True)
            mean_r_avg_365.reset_index(drop=True, inplace=True)
            year=pd.date_range(start='1/1/13', end='12/31/13')
            mean_r_avg_365.index=year
            mean_k_avg_365.index=year
            
            mean_r_avg_365=mean_r_avg_365.resample('MS').sum()
            mean_k_avg_365=mean_k_avg_365.resample('MS').sum()
            
            mean = mean_k_month - mean_r_month

            output_file = current.config["output_path"] + "meteo\\precipitation\\alto_adige\\monthly\\meteo_" \
                + var_str + '_AA_monthly_boxplotSpatialBias_kr11-rea.' + output_format
            createBoxPlot( mean, "Time $[month]$", "Prec. bias $[mm]$", \
                output_file, output_format=output_format, scale_factor=0.5, my_dpi=50 )

            output_file = current.config["output_path"] + "meteo\\precipitation\\alto_adige\\monthly\\meteo_" \
                + var_str + '_AA_monthly_boxplotSpatialBias_kr11-rea_HD.' + output_format
            createBoxPlot( mean, "Time $[month]$", "Prec. bias $[mm]$", \
                output_file, output_format=output_format, scale_factor=0.5, my_dpi=600 )